In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

## Load Airbnb dataset 

In [18]:
columns = ["Country", "City", "Picture Url"]
df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings.csv", usecols=columns, sep=';')
df = df[columns]
df.head(2)

,Country,City,Picture Url
0,Netherlands,Amsterdam,https://public.opendatasoft.com/api/v2/catalog...
1,Netherlands,Amsterdam,https://public.opendatasoft.com/api/v2/catalog...


In [17]:
len(df[df["City"]=="New York City"])

5

In [26]:
df[df["Country"]=="United States"].groupby("City").count()["Country"].reset_index().sort_values("Country", ascending=False)

,City,Country
391,New York,19527
323,Los Angeles,19426
96,Brooklyn,16416
38,Austin,9562
515,San Francisco,8679
622,Washington,7692
508,San Diego,6201
389,New Orleans,5293
138,Chicago,5186
534,Seattle,3812


## Retrieve images from url

In [14]:
for i in range(len(df)):
    image_url = df.iloc[i]["Picture Url"]
    src = "../../data/airbnb-data-science/listing_images/" + str(i) + ".jpg"
    urllib.request.urlretrieve(image_url, src)

KeyboardInterrupt: 

## Get aesthetic score for images

In [ ]:
# https://github.com/idealo/image-quality-assessment
# https://towardsdatascience.com/using-docker-to-set-up-a-deep-learning-environment-on-aws-6af37a78c551
# https://ai.googleblog.com/2017/12/introducing-nima-neural-image-assessment.html
# https://developer.nvidia.com/blog/deep-learning-hotel-aesthetics-photos/
# https://medium.com/idealo-tech-blog/using-deep-learning-to-automatically-rank-millions-of-hotel-images-c7e2d2e5cae2

In [13]:
! ../../image-quality-assessment/predict  --docker-image nima-cpu --base-model-name MobileNet --weights-file /Users/nesara/Documents/aim/cs/projects/image-quality-assessment/models/MobileNet/weights_mobilenet_aesthetic_0.07.hdf5 --image-source /Users/nesara/Documents/aim/cs/projects/data/airbnb-data-science/listing_images/

2020-10-22 17:26:52.218913: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-10-22 17:26:52.229809: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2697945000 Hz
2020-10-22 17:26:52.231269: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x47e7e20 executing computations on platform Host. Devices:
2020-10-22 17:26:52.231849: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
1/1 [==============================] - 3s 3s/step
[
  {
    "image_id": "0",
    "mean_score_prediction": 4.4160667712421855
  },
  {
    "image_id": "1",
    "mean_score_prediction": 5.258465602255001
  },
  {
    "image_id": "2",
    "mean_score_prediction": 5.059895681362832
  },
  {
    "image_id": "3",
    "mean_score_prediction": 5.333361931636318
  },
  {
    "image_id": "4",
    "mean_score_prediction": 4.808626324898796